In [ ]:
! pip install pymongo

You should consider upgrading via the '/Users/erlichsefi/Documents/daily-publish-supermarket-data/venv/bin/python3 -m pip install --upgrade pip' command.


In [99]:
import pymongo
from collections import defaultdict

# Connect to MongoDB (change the URI if needed)
client = pymongo.MongoClient("mongodb://192.168.1.129:27017/")

# Select a database
db = client["supermarket_data"]

# Select a scraper_status_collection
scraper_status_collection = db["ScraperStatus"]
parser_status_collection = db["ParserStatus"]


#o retrieve "timestamp" and "file_name"
query = {}
projection = {"timestamp": 1, "file_name": 1, "_id": 0}  # Retrieve only necessary fields

# Use a dictionary with sets to store unique timestamps per file_name
file_name_dict = defaultdict(set)

# Fetch documents and populate the dictionary
for doc in scraper_status_collection.find(query, projection):
    if "timestamp" in doc and "file_name" in doc:  # Ensure fields exist
        file_name_dict[doc["file_name"]].add(doc["timestamp"])

# Convert sets to sorted lists for better readability
file_name_dict = {file_name: sorted(timestamps) for file_name, timestamps in file_name_dict.items()}

# Print result
for file_name, timestamps in file_name_dict.items():
    print(f"{file_name}: {timestamps}")

keshet: ['20250308030551', '20250308051627', '20250308081936', '20250308111503', '20250308142601', '20250308171416']
yaynotbitan: ['20250308030519', '20250308051623', '20250308081921', '20250308111448', '20250308142545', '20250308171402']
goodpharm: ['20250308031015', '20250308051620', '20250308082852', '20250308111902', '20250308143519', '20250308172009']
hetcohen: ['20250308030547', '20250308051624', '20250308081932', '20250308111455', '20250308142556', '20250308171410']
supersapir: ['20250308031040', '20250308051701', '20250308082446', '20250308111923', '20250308143132', '20250308171758']
mega: ['20250308030725', '20250308051702', '20250308082026', '20250308111552', '20250308142650', '20250308171457']
victory: ['20250308031333', '20250308051711', '20250308082741', '20250308111943', '20250308143153', '20250308171812']
tivtaam: ['20250308031107', '20250308051707', '20250308082540', '20250308111942', '20250308143146', '20250308171811']
yellow: ['20250308031335', '20250308051731', '2025

In [122]:
# Dictionary to store validation results
validation_results = {}
final_validation_results = {}
# Iterate over each chain name and its unique timestamps
for chain_name, timestamps in file_name_dict.items():
    if not timestamps:
        continue  # Skip if there are no timestamps

    validation_results[chain_name] = []
    for sample_timestamp in timestamps:
    
        # Aggregation query to count matching documents
        pipeline = [
            {"$match": {"timestamp": sample_timestamp, "file_name": chain_name}},
            {"$count": "count"}
        ]

        # Execute the aggregation
        result = list(scraper_status_collection.aggregate(pipeline))

        # Check if exactly 4 documents match
        validation_results[chain_name].append(result[0]["count"] == 4 if result else False)
    final_validation_results[chain_name] = all(validation_results[chain_name])


In [108]:
chain_name = 'citymarketkiryatgat'

sample_timestamp = file_name_dict[chain_name][-1]

# find the collecting start
query = {"timestamp": sample_timestamp,"file_name": chain_name, "status":"collected"}
collected_in_this_try = scraper_status_collection.find_one(query)

# find the downloading loads
query = {"timestamp": sample_timestamp,"file_name": chain_name, "status":"downloaded"}
download_in_this_try = scraper_status_collection.find_one(query)




# file saw in the website
files_saw = collected_in_this_try['file_name_collected_from_site']
files_saw = list(map(lambda x: x.replace(".gz",""), files_saw))

# file downloaded
files_downloaded = download_in_this_try['results']
downloaded_file_names = list(map(lambda x: x['file_name'].replace(".gz",""), files_downloaded))


# Diff
saw_but_not_downloaded = set(downloaded_file_names) - set(files_saw)
downloaded_but_not_saw = set(files_saw) - set(downloaded_file_names)



files_saw

['Price7290058266241-045-202503081610',
 'Price7290058266241-046-202503081610',
 'Price7290058266241-046-202503081710']

In [115]:
import datetime
from il_supermarket_scarper import DumpFolderNames

def match_parsing_timestamps(sample_timestamp,chain_name):
    all_parsing_timestamps = list(set(map(lambda x: x['timestamp'],list(parser_status_collection.find({"store_enum":chain_name},{"timestamp":1,"_id":0})))))
    scraping_timestamp = datetime.datetime.strptime(sample_timestamp, "%Y%m%d%H%M%S")
    
    min_delta = None
    associated_stamp = None
    for parsing_timestamp in all_parsing_timestamps:
        diff = datetime.datetime.strptime(parsing_timestamp, "%d%m%Y%H%M%S") - scraping_timestamp
        if min_delta is None or diff < min_delta:
            associated_stamp = parsing_timestamp
            min_delta = diff
    return associated_stamp

def match_chain_names(chain_name):
    for k in DumpFolderNames:
        if k.value.lower() == chain_name.split(".")[0].lower():
            return str(k.name)

matched_chain_name = match_chain_names(chain_name)
matched_timestamp = match_parsing_timestamps(sample_timestamp,matched_chain_name)


In [121]:
from itertools import chain
file_listed_to_parse = list(chain.from_iterable(map(lambda x: map(lambda y: y.replace(".aspx.xml",""),x['response']['files_to_process']), parser_status_collection.find({"store_enum":matched_chain_name,"timestamp":matched_timestamp}))))
file_listed_to_parse

['Price7290058266241-045-202503081610',
 'Price7290058266241-046-202503081610',
 'Price7290058266241-046-202503081710']

In [117]:
parsing_results = list(chain.from_iterable(map(lambda x: x['response']['execution_log'], parser_status_collection.find({"store_enum":matched_chain_name,"timestamp":matched_timestamp}))))
parsing_results


[{'status': True,
  'store_folder': 'app_data/dumps/CityMarketKiryatGat',
  'file_name': 'Price7290058266241-045-202503081610.aspx.xml',
  'prefix_file_name': 'Price7290058266241',
  'extracted_store_number': '045',
  'extracted_chain_id': '7290058266241',
  'extracted_date': '2025-03-08 16:10:00',
  'detected_filetype': 'PRICE_FILE'},
 {'status': True,
  'store_folder': 'app_data/dumps/CityMarketKiryatGat',
  'file_name': 'Price7290058266241-046-202503081610.aspx.xml',
  'prefix_file_name': 'Price7290058266241',
  'extracted_store_number': '046',
  'extracted_chain_id': '7290058266241',
  'extracted_date': '2025-03-08 16:10:00',
  'detected_filetype': 'PRICE_FILE'},
 {'status': True,
  'store_folder': 'app_data/dumps/CityMarketKiryatGat',
  'file_name': 'Price7290058266241-046-202503081710.aspx.xml',
  'prefix_file_name': 'Price7290058266241',
  'extracted_store_number': '046',
  'extracted_chain_id': '7290058266241',
  'extracted_date': '2025-03-08 17:10:00',
  'detected_filetype': '